In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')

import lightgbm as lgb
from sklearn.metrics import r2_score

In [2]:
spark_train = spark.read.format("csv").option("inferSchema", "true").option("header","true").load("dbfs:/mnt/data/train.csv").limit(100000)
spark_test = spark.read.format("csv").option("inferSchema", "true").option("header","true").load("dbfs:/mnt/data/test.csv")

In [3]:
train = spark_train.toPandas()
test = spark_test.toPandas()

In [4]:
train.describe()

Out[ 12 ]: 
 fare_amount pickup_longitude pickup_latitude dropoff_longitude \
count 100000.000000 100000.000000 100000.000000 100000.000000 
mean 11.354652 -72.494682 39.914481 -72.490967 
std 9.716777 10.693934 6.225686 10.471386 
min -44.900000 -736.550000 -74.007670 -84.654241 
25% 6.000000 -73.992041 40.734996 -73.991215 
50% 8.500000 -73.981789 40.752765 -73.980000 
75% 12.500000 -73.966982 40.767258 -73.963433 
max 200.000000 40.787575 401.083332 40.851027 

 dropoff_latitude passenger_count 
count 100000.000000 100000.000000 
mean 39.919053 1.673820 
std 6.213427 1.300171 
min -74.006377 0.000000 
25% 40.734182 1.000000 
50% 40.753243 1.000000 
75% 40.768166 2.000000 
max 404.616667 6.000000

In [5]:
test.describe()

Out[ 13 ]: 
 pickup_longitude pickup_latitude dropoff_longitude dropoff_latitude \
count 9914.000000 9914.000000 9914.000000 9914.000000 
mean -73.974722 40.751041 -73.973657 40.751743 
std 0.042774 0.033541 0.039072 0.035435 
min -74.252193 40.573143 -74.263242 40.568973 
25% -73.992501 40.736125 -73.991247 40.735254 
50% -73.982326 40.753051 -73.980015 40.754065 
75% -73.968013 40.767113 -73.964059 40.768757 
max -72.986532 41.709555 -72.990963 41.696683 

 passenger_count 
count 9914.000000 
mean 1.671273 
std 1.278747 
min 1.000000 
25% 1.000000 
50% 1.000000 
75% 2.000000 
max 6.000000

In [6]:
print('Old size: {}'.format(len(train)))
train = train[train.fare_amount>=0]
print('New size: {}'.format(len(train)))

print(train.isnull().sum())
print('Old size: {}'.format(len(train)))
train = train.dropna(how = 'any', axis = 'rows')
print('New size: {}'.format(len(train)))

print(test.isnull().sum())
print('Old size: {}'.format(len(test)))
test = test.dropna(how = 'any', axis = 'rows')
print('New size: {}'.format(len(test)))

Old size: 100000
New size: 99991
key 0
fare_amount 0
pickup_datetime 0
pickup_longitude 0
pickup_latitude 0
dropoff_longitude 0
dropoff_latitude 0
passenger_count 0
dtype: int64
Old size: 99991
New size: 99991
key 0
pickup_datetime 0
pickup_longitude 0
pickup_latitude 0
dropoff_longitude 0
dropoff_latitude 0
passenger_count 0
dtype: int64
Old size: 9914
New size: 9914

In [7]:
data = pd.concat([train, test])

In [8]:
plt.figure(figsize = (12, 4))
sns.heatmap(data.isnull(), yticklabels=False, cbar=False, cmap='viridis')
plt.grid(True)
plt.xticks(rotation=10)
display(plt.show())

In [9]:
data['passenger_count'].describe()

Out[ 17 ]: 
count 109905.000000
mean 1.673536
std 1.298167
min 0.000000
25% 1.000000
50% 1.000000
75% 2.000000
max 6.000000
Name: passenger_count, dtype: float64

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109905 entries, 0 to 9913
Data columns (total 8 columns):
dropoff_latitude 109905 non-null float64
dropoff_longitude 109905 non-null float64
fare_amount 99991 non-null float64
key 109905 non-null object
passenger_count 109905 non-null int32
pickup_datetime 109905 non-null object
pickup_latitude 109905 non-null float64
pickup_longitude 109905 non-null float64
dtypes: float64(5), int32(1), object(2)
memory usage: 7.1+ MB

In [11]:
data['key'] = pd.to_datetime(data['key'])
data['pickup_datetime']  = pd.to_datetime(data['pickup_datetime'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109905 entries, 0 to 9913
Data columns (total 8 columns):
dropoff_latitude 109905 non-null float64
dropoff_longitude 109905 non-null float64
fare_amount 99991 non-null float64
key 109905 non-null datetime64[ns]
passenger_count 109905 non-null int32
pickup_datetime 109905 non-null datetime64[ns]
pickup_latitude 109905 non-null float64
pickup_longitude 109905 non-null float64
dtypes: datetime64[ns](2), float64(5), int32(1)
memory usage: 7.1 MB

In [12]:
#extract year
data["year"]  = pd.DatetimeIndex(data["pickup_datetime"]).year
#extract month
data["month"] = pd.DatetimeIndex(data["pickup_datetime"]).month
#extract day 
data["day"] = data["pickup_datetime"].dt.day
#extract hour
data["hour"] = data["pickup_datetime"].dt.hour 

In [13]:
data.columns

Out[ 21 ]: 
Index(['dropoff_latitude', 'dropoff_longitude', 'fare_amount', 'key',
 'passenger_count', 'pickup_datetime', 'pickup_latitude',
 'pickup_longitude', 'year', 'month', 'day', 'hour'],
 dtype='object')

In [14]:
#replace 0's in coordinates with null values
coord = ['pickup_longitude','pickup_latitude', 
         'dropoff_longitude', 'dropoff_latitude']

# for i in coord:
#   data = data[(data[i] > data[i].quantile(.1)) & (data[i] < data[i].quantile(.8))]

for i in coord:
    data[i] = data[i].replace(0,np.nan)
    data = data[data[i].notnull()]

#radius of earth in kilometers
R = 6373.0

pickup_lat  = np.radians(data["pickup_latitude"])
pickup_lon  = np.radians(data["pickup_longitude"])
dropoff_lat = np.radians(data["dropoff_latitude"])
dropoff_lon = np.radians(data["dropoff_longitude"])

dist_lon = dropoff_lon - pickup_lon
dist_lat = dropoff_lat - pickup_lat

#Formula
a = (np.sin(dist_lat/2))**2 + np.cos(pickup_lat) * np.cos(dropoff_lat) * (np.sin(dist_lon/2))**2 
c = 2 * np.arctan2( np.sqrt(a), np.sqrt(1-a) ) 
d = R * c #(where R is the radius of the Earth)

data["trip_distance_km"] = d

data[coord + ["trip_distance_km"]].head(7)

Out[ 22 ]: 
 pickup_longitude pickup_latitude dropoff_longitude dropoff_latitude \
0 -73.844311 40.721319 -73.841610 40.712278 
1 -74.016048 40.711303 -73.979268 40.782004 
2 -73.982738 40.761270 -73.991242 40.750562 
3 -73.987130 40.733143 -73.991567 40.758092 
4 -73.968095 40.768008 -73.956655 40.783762 
5 -74.000964 40.731630 -73.972892 40.758233 
6 -73.980002 40.751662 -73.973802 40.764842 

 trip_distance_km 
0 1.031088 
1 8.452786 
2 1.389961 
3 2.800149 
4 1.999784 
5 3.788428 
6 1.556295

In [15]:
data["trip_distance_km"].describe()

Out[ 23 ]: 
count 107900.000000
mean 4.052849
std 59.275774
min 0.000000
25% 1.259469
50% 2.164290
75% 3.944056
max 6025.780800
Name: trip_distance_km, dtype: float64

In [16]:
data = data[(data["trip_distance_km"] < 200) | (data["fare_amount"].isnull())].copy()

In [17]:
data["trip_distance_km"].describe()

Out[ 25 ]: 
count 107882.000000
mean 3.375236
std 4.218690
min 0.000000
25% 1.259401
50% 2.163851
75% 3.942234
max 174.862983
Name: trip_distance_km, dtype: float64

In [18]:
m_train = data[data["fare_amount"].isnull() == False].copy()
m_train.shape

Out[ 26 ]: (97968, 13)

In [19]:
m_test = data[data["fare_amount"].isnull() == True].copy()
m_test.shape

Out[ 27 ]: (9914, 13)

In [20]:
# Not including the pickup_datetime columns as datetime columns cannot be directly used while modelling. Features need to extracted from the 
# timestamp fields which will later be used as features for modelling.
m_train = m_train.drop(['key','pickup_datetime'], axis = 1)
m_test = m_test.drop(['key','pickup_datetime'], axis = 1)

In [21]:
x_train = m_train.drop("fare_amount", axis=1)
y_train = m_train['fare_amount']
x_test = m_test.drop("fare_amount", axis=1)

In [22]:
x_train.shape, y_train.shape, test.shape

Out[ 30 ]: ((97968, 10), (97968,), (9914, 7))

In [23]:
x_test.loc[2039,"trip_distance_km"] = 7.71
x_test.loc[2486,"trip_distance_km"] = 7.71

In [24]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GridSearchCV
# rf = RandomForestRegressor()
# rf.fit(x_train, y_train)
# rf_predict = rf.predict(x_test)

# param_grid = {
    #'bootstrap': [True],
    #'max_depth': [80, 90, 100, 110],
    #'max_features': [5, 7],
    #'min_samples_leaf': [300, 400, 50],
    #'min_samples_split': [80, 100, 120],
    #'n_estimators': [100, 200, 300, 1000]}

#grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, verbose = 2)
# # Fit the grid search to the data
#grid_search.fit(x_train, y_train)

In [25]:
# grid_search.best_params_

In [26]:
#rf_predict = grid_search.predict(x_test)

In [27]:
#  submission = spark.read.format("csv").option("inferSchema", "true").option("header","true").load("dbfs:/mnt/data/sample_submission.csv")
# .option("inferSchema", "true") - преобразовывает колонку key к типу DateTime => значение миллисекунд теряется => получаем много дубликатов.
# Следовательно, данную опцию не следует использовать в момент чтения файла
# submission = spark.read.format("csv").option("header","true").load("dbfs:/mnt/data/sample_submission.csv")
# submission_pd = submission.toPandas()
# submission_pd['fare_amount'] = rf_predict
# submission_pd.head(20)

In [28]:
# submission_pd_spark = spark.createDataFrame(submission_pd)
# submission_pd_spark.repartition(1).write.format("com.databricks.spark.csv").option("header", "true").csv("dbfs:/mnt/data/submission_6.csv")

##### BOOSTING USING LGB

In [31]:
params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': -1,
        'verbose': 0,
        'num_leaves': 31,
        'learning_rate': 0.05,
        'max_depth': -1,
        'subsample': 0.8,
        'subsample_freq': 1,
        'colsample_bytree': 0.6,
        'reg_aplha': 1,
        'reg_lambda': 0.001,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10,
        'scale_pos_weight':1     
    }

In [32]:
pred_test_y = np.zeros(x_test.shape[0])
pred_test_y.shape

Out[ 38 ]: (9914,)

In [33]:
import lightgbm as lgb
train_set = lgb.Dataset(x_train, y_train, silent=True)
train_set

Out[ 39 ]: <lightgbm.basic.Dataset at 0x7fb6172b5f28>

In [34]:
model = lgb.train(params, train_set = train_set, num_boost_round=300)

In [35]:
print(model)

<lightgbm.basic.Booster object at 0x7fb6185466a0>

In [36]:
pred_test_y = model.predict(x_test, num_iteration = model.best_iteration)

In [37]:
print(pred_test_y)

[ 10.49882591 10.52498707 4.84754309 ..., 55.77153869 21.32779195
 7.61176687]